## Working with Snapshot camera

Idea: find only the large cluster to reduce detection noise. Then, use the lifespan of customer's cluster to find the most probable vehicle in the snapshot.
Snapshot cam: takes photo every 1:15 second

In [35]:
import sys

sys.path.append("/home/jeans/internship/parking-customer-count")

from countpassenger.Config import conf
from countpassenger import Preprocess
from countpassenger import Cluster
from countpassenger import Utils

import countpassenger
import pandas as pd
import numpy as np

import os.path as osp
import os

start = 1714284000
stop= 1714288000


pd.set_option("display.precision", 10)
CURRENT_CAM = "mbk-14-12"

df_vehicle = pd.read_csv(
    osp.join(conf.RESOURCES_RAW_DIR, "2024-04-28/mbk-tourist-vehicle-object-20240428.csv")
)
df_cross = pd.read_csv(
    osp.join(conf.RESOURCES_RAW_DIR, "2024-04-28/mbk-tourist-raw-cross-object-20240428.csv")
)
df_reverse = pd.read_csv(
    osp.join(conf.RESOURCES_RAW_DIR, "2024-04-28/mbk-tourist-raw-reverse-object-20240428.csv")
)

df_vehicle = Preprocess.df_clean_vehicle(
    df_vehicle=df_vehicle,
)
df_vehicle = Preprocess.filter_camera(df_vehicle, camera_name=CURRENT_CAM + "-vehicle-snapshot")
df_vehicle = Preprocess.sort_df(df_vehicle, conf.VEHICLE_INTEREST_NON_SNAPSHOT)

print(df_vehicle[[*conf.VEHICLE_INTEREST_SNAPSHOT]].to_string())

                          camera       timestamp_precise  timestamp_unix  timestamp_unix_end vehicle_type    xmid    ymid plate_number
2325  mbk-14-12-vehicle-snapshot 2024-04-28 02:19:58.762      1714270798          1714271059       pickup   845.5   252.5          NaN
2313  mbk-14-12-vehicle-snapshot 2024-04-28 02:27:30.084      1714271250          1714271324       pickup  1870.5    51.5       327722
2314  mbk-14-12-vehicle-snapshot 2024-04-28 02:28:45.064      1714271325          1714271362        truck  1095.0   687.0          NaN
2299  mbk-14-12-vehicle-snapshot 2024-04-28 02:29:58.802      1714271398          1714271434        truck  1093.5   683.5          NaN
2301  mbk-14-12-vehicle-snapshot 2024-04-28 02:30:37.634      1714271437          1714271473        truck  1095.0   686.0          NaN
2300  mbk-14-12-vehicle-snapshot 2024-04-28 02:31:52.613      1714271512          1714271549        truck  1095.0   686.0          NaN
2302  mbk-14-12-vehicle-snapshot 2024-04-28 02:33:07.59

In [36]:
df_cross = Preprocess.df_clean_customer(df_cross)
df_cross = Preprocess.filter_camera(df_cross, camera_name=CURRENT_CAM)
df_cross = Preprocess.sort_df(df_cross, conf.CUSTOMER_INTEREST_SNAPSHOT)

# print(df_cross[[*conf.CUSTOMER_INTEREST_SNAPSHOT,'timestamp_unix']].to_string())

df_reverse = Preprocess.df_clean_customer(df_reverse)
df_reverse = Preprocess.filter_camera(df_reverse, camera_name=CURRENT_CAM)
df_reverse = Preprocess.sort_df(df_reverse, conf.CUSTOMER_INTEREST_SNAPSHOT)

# print(df_reverse[[*conf.CUSTOMER_INTEREST_SNAPSHOT, "timestamp_unix"]].to_string())

### Cluster only large vehicle 

In [37]:
params = dict(
    min_cluster_size=2, #have at least 5 people together
    min_samples=3,
    cluster_selection_epsilon=0, 
    max_cluster_size=30,
    alpha=1.0,
    algorithm="brute",
    n_jobs=None,
    cluster_selection_method="eom", # eom with minsample = 3 is kinda good
    store_centers=None,
)

df_cross, cluster_cross = Cluster.perform_cross_clustering(df_cross=df_cross,params=params)
print(Preprocess.sort_df(cluster_cross, conf.CLUSTER_INTEREST)[conf.CLUSTER_INTEREST][(cluster_cross["timestamp_unix_max"] >= start) & (cluster_cross["timestamp_unix_min"] <= stop)].to_string())

/home/jeans/internship/parking-customer-count/countpassenger/Cluster.py:60: RuntimeWarning: overflow encountered in exp
  exponential_part = np.exp(x - n) + n - 1
/home/jeans/internship/parking-customer-count/countpassenger/Cluster.py:58: RuntimeWarning: overflow encountered in exp
  sigmoid = 1 / (1 + np.exp(-k * (x - n)))


          timestamp_min        timestamp_max  count  cluster_id        xmid_mean       ymid_mean
0   2024-04-28 02:00:08  2024-04-28 15:31:18     66          -1  1619.5303030303  336.7954545455
69  2024-04-28 05:59:01  2024-04-28 06:01:53      5          68   832.8000000000  612.0000000000
57  2024-04-28 06:06:11  2024-04-28 06:08:34      6          56  1460.7500000000  461.8333333333
76  2024-04-28 06:11:32  2024-04-28 06:12:24      5          75  2065.5000000000  142.9000000000
77  2024-04-28 06:12:42  2024-04-28 06:13:36      6          76  1164.1666666667  638.4166666667
58  2024-04-28 06:16:23  2024-04-28 06:16:59      3          57   495.1666666667  837.5000000000
78  2024-04-28 06:19:02  2024-04-28 06:19:12      3          77  1117.1666666667  644.3333333333
79  2024-04-28 06:19:48  2024-04-28 06:20:11      3          78  1140.0000000000  552.8333333333
59  2024-04-28 06:29:17  2024-04-28 06:29:23      4          58  1962.6250000000  172.3750000000
70  2024-04-28 06:30:14  2024-

/home/jeans/anaconda3/envs/ml_env/lib/python3.12/site-packages/sklearn/cluster/_hdbscan/hdbscan.py:264: UserWarning: The minimum spanning tree contains edge weights with value infinity. Potentially, you are missing too many distances in the initial distance matrix for the given neighborhood size.
  warn(
/tmp/ipykernel_166320/4192676930.py:19: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  print(Preprocess.sort_df(cluster_cross, conf.CLUSTER_INTEREST)[conf.CLUSTER_INTEREST][(cluster_cross["timestamp_unix_max"] >= start) & (cluster_cross["timestamp_unix_min"] <= stop)].to_string())


In [42]:
params = dict(
    min_cluster_size=2, #have at least 5 people together
    min_samples=None,
    cluster_selection_epsilon=0,
    max_cluster_size=30,
    alpha=1.0,
    algorithm="brute",
    cluster_selection_method="leaf",
    store_centers=None,
)

df_reverse, cluster_reverse = Cluster.perform_reverse_clustering(df_reverse=df_reverse,params=params)
print(Preprocess.sort_df(cluster_reverse, conf.CLUSTER_INTEREST)[(cluster_reverse["timestamp_unix_max"] >= start) & (cluster_reverse["timestamp_unix_min"] <= stop)][conf.CLUSTER_INTEREST].to_string())

/home/jeans/internship/parking-customer-count/countpassenger/Cluster.py:60: RuntimeWarning: overflow encountered in exp
  exponential_part = np.exp(x - n) + n - 1
/home/jeans/internship/parking-customer-count/countpassenger/Cluster.py:58: RuntimeWarning: overflow encountered in exp
  sigmoid = 1 / (1 + np.exp(-k * (x - n)))


           timestamp_min        timestamp_max  count  cluster_id        xmid_mean       ymid_mean
0    2024-04-28 03:17:35  2024-04-28 16:22:51     33          -1  1571.9696969697  352.2878787879
46   2024-04-28 06:02:19  2024-04-28 06:06:06      2          45  1302.2500000000  436.0000000000
47   2024-04-28 06:11:10  2024-04-28 06:14:23      3          46  2108.3333333333   63.0000000000
56   2024-04-28 06:17:11  2024-04-28 06:18:15      4          55  1675.7500000000  290.5000000000
57   2024-04-28 06:19:04  2024-04-28 06:19:13      3          56  1374.3333333333  377.0000000000
20   2024-04-28 06:39:36  2024-04-28 06:41:35      2          19  2052.2500000000  113.5000000000
37   2024-04-28 06:45:42  2024-04-28 06:48:23      3          36  1431.0000000000  364.0000000000
38   2024-04-28 06:52:26  2024-04-28 06:57:28      6          37  1312.0000000000  408.1666666667
130  2024-04-28 07:06:35  2024-04-28 07:06:41      3         129   880.8333333333  863.6666666667


/home/jeans/anaconda3/envs/ml_env/lib/python3.12/site-packages/sklearn/cluster/_hdbscan/hdbscan.py:264: UserWarning: The minimum spanning tree contains edge weights with value infinity. Potentially, you are missing too many distances in the initial distance matrix for the given neighborhood size.
  warn(
/tmp/ipykernel_166320/3930326985.py:17: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  print(Preprocess.sort_df(cluster_reverse, conf.CLUSTER_INTEREST)[conf.CLUSTER_INTEREST][(cluster_reverse["timestamp_unix_max"] >= start) & (cluster_reverse["timestamp_unix_min"] <= stop)].to_string())


lifespan of customer's cluster to find the most probable vehicle in the snapshot.
Snapshot cam: takes photo every 1:15 second

nearest x,y ? in time windows?